# Tutorial 05: weak imposition of Dirichlet BCs by a Lagrange multiplier (inf-sup computation)

In this tutorial we compute the inf-sup constant of the saddle point problem resulting from the discretization of the following Laplace problem
$$\begin{cases}
-\Delta u = f, & \text{in } \Omega,\\
 u   = g, & \text{on } \Gamma = \partial\Omega,
\end{cases}$$

where $\Omega$ is the unit ball in 2D, and for which the non-homogeneous Dirichlet boundary conditions are imposed by a Lagrange multiplier.

The resulting eigenvalue problem is
$$
\text{find } \eta, u, \lambda \in \mathbb{R} \times V \times M \text{ s.t. }\\
\begin{cases}
\int_\Omega \nabla u \cdot \nabla v + \int_\Gamma \lambda v = 0, & \forall v \in V,\\
\int_\Gamma u \mu = \eta \int_\Gamma \lambda \mu, & \forall \mu \in M
\end{cases}
$$
where
$$
V = H^1(\Omega),\\
M = L^{2}(\Gamma).\\
$$

In [ ]:
from numpy import arange, isclose, sqrt, where
from petsc4py import PETSc
from ufl import grad, inner, Measure
from dolfinx import *
from dolfinx.cpp.mesh import GhostMode
from dolfinx.fem import locate_dofs_topological
from dolfinx.io import XDMFFile
from multiphenics import *
from multiphenics.fem import block_assemble

### Mesh

In [ ]:
with XDMFFile(MPI.comm_world, "data/circle.xdmf") as infile:
    mesh = infile.read_mesh(GhostMode.none)
with XDMFFile(MPI.comm_world, "data/circle_subdomains.xdmf") as infile:
    subdomains = infile.read_mf_size_t(mesh)
with XDMFFile(MPI.comm_world, "data/circle_boundaries.xdmf") as infile:
    boundaries = infile.read_mf_size_t(mesh)
facets_Gamma = where(boundaries.values == 1)[0]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

### Eigenvalue problem

In [ ]:
# Define a block function space
V = FunctionSpace(mesh, ("Lagrange", 2))
dofs_V = arange(0, V.dofmap.index_map.block_size*(V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts))
dofs_V_Gamma = locate_dofs_topological(V, boundaries.dim, facets_Gamma)
W = BlockFunctionSpace([V, V], restrict=[dofs_V, dofs_V_Gamma])

In [ ]:
# Define trial and test functions
ul = BlockTrialFunction(W)
(u, l) = block_split(ul)
vm = BlockTestFunction(W)
(v, m) = block_split(vm)

In [ ]:
# Define problem block forms
a = [[inner(grad(u), grad(v))*dx, - l*v*ds],
     [- u*m*ds                  , 0       ]]
b = [[0                         , 0       ],
     [0                         , - l*m*ds]]
a = BlockForm2(a, [W, W])
b = BlockForm2(b, [W, W])
A = block_assemble(a)
B = block_assemble(b)

In [ ]:
# Solve
options = PETSc.Options()
options_prefix = "multiphenics_eigensolver_"
options.setValue(options_prefix + "eps_gen_non_hermitian", "")
options.setValue(options_prefix + "eps_target_real", "")
options.setValue(options_prefix + "eps_target", 1.e-5)
options.setValue(options_prefix + "st_type", "sinvert")
options.setValue(options_prefix + "st_ksp_type", "preonly")
options.setValue(options_prefix + "st_pc_type", "lu")
options.setValue(options_prefix + "st_pc_factor_mat_solver_type", "mumps")
eigensolver = BlockSLEPcEigenSolver(A, B)
eigensolver.set_options_prefix(options_prefix)
eigensolver.set_from_options()
eigensolver.solve(1)
eigv = eigensolver.get_eigenvalue(0)
r, c = eigv.real, eigv.imag
assert abs(c) < 1.e-10
assert r > 0., "r = " + str(r) + " is not positive"
print("Inf-sup constant: ", sqrt(r))
assert isclose(sqrt(r), 0.125473)